# Chronicle Flood Database Intersection with MERIT Hydro dataset

This notebook provides a comprehensive analysis of the Chronicle urban flood dataset, containing over 880,000 flood events worldwide from 2000-2025.


## Hydro-merit upa index:

In [4]:
# ============================================
# 0) IMPORTS
# ============================================
import os
import glob
import math
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
import ee
import geemap

In [2]:
# ============================================
# 1) CONFIGURATION
# ============================================
PKL_PATH = r"D:\Development\RESEARCH\urban_flood_database\chronicle\chronicle_urban_df.pkl"
GEOM_WKT_COL = "geometry_wkt"

# Earth Engine dataset
MERIT_IMAGE_ID = "MERIT/Hydro/v1_0_1"
UPA_BAND = "upa"  # upstream drainage area (km^2)

# Batch settings
BATCH_SIZE = 50
N_BATCHES_TO_RUN = 5000  # Will run 30 NEW batches

# Zonal stats settings
SCALE_M = 90

# Output settings
OUT_DIR = r"D:\Development\RESEARCH\urban_flood_database\chronicle\hydromerit_pluvial_outputs"

# The final output filenames (PKL)
OUT_STATS_PKL_ALL = os.path.join(OUT_DIR, "pfdi_stats_ALL_batches.pkl")
OUT_ENRICHED_PKL = os.path.join(OUT_DIR, "chronicle_df_with_pfdi_FULL.pkl")

In [3]:
# ============================================
# 2) HELPERS
# ============================================
def ensure_out_dir(path):
    """Create output directory if it doesn't exist."""
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def fix_geometry_safe(geom):
    """Fix minor geometry issues; returns original geometry if fix fails."""
    try:
        if geom is None:
            return None
        return geom.buffer(0)
    except Exception:
        return geom


def add_event_id(df_in):
    """Add stable integer event_id based on the DataFrame index."""
    df_out = df_in.copy()
    df_out["event_id"] = df_out.index.astype(int)
    return df_out


def compute_area_km2(gdf_wgs84):
    """Compute polygon area in km^2 using an equal-area CRS."""
    gdf_area = gdf_wgs84.to_crs("EPSG:6933").copy()
    return (gdf_area.geometry.area / 1_000_000.0).astype(float)


def ee_fc_to_df(fc):
    """Convert an EE FeatureCollection to a pandas DataFrame with robust fallbacks."""
    if hasattr(geemap, "ee_to_df"):
        return geemap.ee_to_df(fc)

    if hasattr(geemap, "ee_to_pandas"):
        return geemap.ee_to_pandas(fc)

    # Fallback: pull features client-side (slower, but robust for small batches)
    features = fc.getInfo().get("features", [])
    rows = [f.get("properties", {}) for f in features]
    return pd.DataFrame(rows)


def initialize_ee():
    """Initialize Earth Engine; authenticate interactively if needed."""
    try:
        ee.Initialize()
        print("Earth Engine initialized.")
    except Exception:
        print("Earth Engine not initialized. Running authentication...")
        ee.Authenticate()
        ee.Initialize()
        print("Earth Engine authenticated and initialized.")


def pick_stat_column(columns, preferred, fallback):
    """Pick a column name from EE output (supports with/without band prefix)."""
    if preferred in columns:
        return preferred
    if fallback in columns:
        return fallback
    return None


def get_start_params_fast(out_dir):
    """
    Finds the last batch file, reads ONLY that file, 
    and determines the next row index based on the max event_id.
    This ensures continuity even if batch sizes change.
    """
    if not os.path.exists(out_dir):
        return 0, 0
    
    # 1. Find all PKL files
    pattern = os.path.join(out_dir, "pfdi_stats_batch_*.pkl")
    files = glob.glob(pattern)
    
    if not files:
        return 0, 0
    
    # 2. Identify the file with the highest batch number
    max_batch = -1
    last_file_path = None
    
    for f in files:
        try:
            filename = os.path.basename(f)
            # filename format: pfdi_stats_batch_XXXX.pkl
            num_part = filename.split('_')[-1].split('.')[0]
            batch_num = int(num_part)
            
            if batch_num > max_batch:
                max_batch = batch_num
                last_file_path = f
        except ValueError:
            continue
            
    if last_file_path is None:
        return 0, 0

    print(f"Checking last saved file for continuity: {os.path.basename(last_file_path)}")
    
    # 3. Read ONLY the last file to find the last processed event_id
    try:
        df_last = pd.read_pickle(last_file_path)
        if "event_id" in df_last.columns and not df_last.empty:
            last_event_id = df_last["event_id"].max()
            # The next row index corresponds to event_id + 1 (assuming event_id matches index)
            next_row_idx = int(last_event_id) + 1
        else:
            print("Warning: Last file was empty or missing event_id. Starting manual check recommended.")
            # Fallback: assume we just continue batch numbering but start index is unknown (risk of overlap/gap)
            return max_batch + 1, 0 
            
    except Exception as e:
        print(f"Error reading last file: {e}")
        return max_batch + 1, 0

    return max_batch + 1, next_row_idx

In [1]:
# ============================================
# 3) AUTHENTICATE & INITIALIZE EARTH ENGINE
# ============================================
initialize_ee()


# ============================================
# 4) LOAD PKL AND PREPARE GEODATAFRAME
# ============================================
ensure_out_dir(OUT_DIR)

df = pd.read_pickle(PKL_PATH)
df = add_event_id(df)

if GEOM_WKT_COL not in df.columns:
    raise ValueError(f"Column '{GEOM_WKT_COL}' not found. Available columns: {list(df.columns)}")

gdf = gpd.GeoDataFrame(
    df,
    geometry=df[GEOM_WKT_COL].apply(wkt.loads),
    crs="EPSG:4326"
)

gdf["geometry"] = gdf["geometry"].apply(fix_geometry_safe)
gdf = gdf[gdf.geometry.notnull()].copy()

gdf_min = gdf[["event_id", "geometry"]].copy()
gdf_min["poly_area_km2"] = compute_area_km2(gdf_min)

print("Chronicle GeoDataFrame loaded.")
print("Total Rows in Dataset:", len(gdf_min))


# ============================================
# 5) LOAD MERIT HYDRO
# ============================================
merit = ee.Image(MERIT_IMAGE_ID)
upa = merit.select(UPA_BAND)

# ============================================
# 6) DEFINE REDUCER
# ============================================
reducer = ee.Reducer.max().combine(
    ee.Reducer.percentile([95, 99]),
    sharedInputs=True
)


# ============================================
# 7) RUN NEW BATCHES (SMART RESUME)
# ============================================
# Get start params using the optimized method
next_batch_id, start_row_idx = get_start_params_fast(OUT_DIR)
n_rows = len(gdf_min)

print(f"--- SMART AUTO-DETECT ---")
print(f"Next Batch File ID: {next_batch_id}")
print(f"Last Processed Event ID : {start_row_idx - 1}")
print(f"Resuming from Row Index: {start_row_idx}")
print(f"Planning to run {N_BATCHES_TO_RUN} new batches of size {BATCH_SIZE}.")

# Initialize manual counters
current_row_idx = start_row_idx
current_batch_id = next_batch_id

# batch_outputs = []

# Loop for the requested number of batches
for _ in range(N_BATCHES_TO_RUN):
    
    # Define start and end rows for THIS batch
    i0 = current_row_idx
    i1 = min(current_row_idx + BATCH_SIZE, n_rows)

    # Stop if we have exceeded the dataset size
    if i0 >= n_rows:
        print("Reached end of dataset. Stopping.")
        break

    print(f"Processing Batch {current_batch_id} (Rows {i0} - {i1})...")

    gdf_batch = gdf_min.iloc[i0:i1].copy()

    # Convert batch polygons to EE FeatureCollection
    fc = geemap.geopandas_to_ee(gdf_batch[["event_id", "geometry"]])

    # Compute zonal stats
    stats_fc = upa.reduceRegions(
        collection=fc,
        reducer=reducer,
        scale=SCALE_M,
        tileScale=8
    )

    df_stats = ee_fc_to_df(stats_fc)

    # Merge local area
    df_area = gdf_batch[["event_id", "poly_area_km2"]].copy()
    out = df_stats.merge(df_area, on="event_id", how="left")

    # Handle EE output column naming differences
    max_col = pick_stat_column(out.columns, "upa_max", "max")
    p95_col = pick_stat_column(out.columns, "upa_p95", "p95")
    p99_col = pick_stat_column(out.columns, "upa_p99", "p99")

    if p95_col is None:
        print(f"Warning: Batch {current_batch_id} returned empty or invalid stats.")
        # Advance counters even if failed, to avoid infinite loop
        current_row_idx += BATCH_SIZE
        current_batch_id += 1
        continue

    # Compute PFDI with explicit name
    out["PFDI_p95"] = out[p95_col] / out["poly_area_km2"]

    # Standardize column names
    rename_map = {}
    if max_col is not None:
        rename_map[max_col] = "upa_max"
    if p95_col is not None:
        rename_map[p95_col] = "upa_p95"
    if p99_col is not None:
        rename_map[p99_col] = "upa_p99"
    out = out.rename(columns=rename_map)

    # Keep a clean set of columns
    keep_cols = ["event_id", "poly_area_km2", "upa_max", "upa_p95", "upa_p99", "PFDI_p95"]
    keep_cols = [c for c in keep_cols if c in out.columns]
    out = out[keep_cols].copy()

    # Save partial result immediately as PKL
    partial_path = os.path.join(OUT_DIR, f"pfdi_stats_batch_{current_batch_id:04d}.pkl")
    out.to_pickle(partial_path)
    
#     batch_outputs.append(out)

    print(f"Finished batch {current_batch_id}. Saved: {os.path.basename(partial_path)}")

# Optional: Force clean up to be super safe
    del out, df_stats, fc
    import gc
    gc.collect()
    
    # Update counters for next iteration
    current_row_idx += BATCH_SIZE
    current_batch_id += 1

Earth Engine initialized.
Chronicle GeoDataFrame loaded.
Total Rows in Dataset: 882972
Checking last saved file for continuity: pfdi_stats_batch_10437.pkl
--- SMART AUTO-DETECT ---
Next Batch File ID: 10438
Last Processed Event ID : 833069
Resuming from Row Index: 833070
Planning to run 5000 new batches of size 50.
Processing Batch 10438 (Rows 833070 - 833120)...
Finished batch 10438. Saved: pfdi_stats_batch_10438.pkl
Processing Batch 10439 (Rows 833120 - 833170)...
Finished batch 10439. Saved: pfdi_stats_batch_10439.pkl
Processing Batch 10440 (Rows 833170 - 833220)...
Finished batch 10440. Saved: pfdi_stats_batch_10440.pkl
Processing Batch 10441 (Rows 833220 - 833270)...
Finished batch 10441. Saved: pfdi_stats_batch_10441.pkl
Processing Batch 10442 (Rows 833270 - 833320)...
Finished batch 10442. Saved: pfdi_stats_batch_10442.pkl
Processing Batch 10443 (Rows 833320 - 833370)...
Finished batch 10443. Saved: pfdi_stats_batch_10443.pkl
Processing Batch 10444 (Rows 833370 - 833420)...
Fini

Finished batch 10513. Saved: pfdi_stats_batch_10513.pkl
Processing Batch 10514 (Rows 836870 - 836920)...
Finished batch 10514. Saved: pfdi_stats_batch_10514.pkl
Processing Batch 10515 (Rows 836920 - 836970)...
Finished batch 10515. Saved: pfdi_stats_batch_10515.pkl
Processing Batch 10516 (Rows 836970 - 837020)...
Finished batch 10516. Saved: pfdi_stats_batch_10516.pkl
Processing Batch 10517 (Rows 837020 - 837070)...
Finished batch 10517. Saved: pfdi_stats_batch_10517.pkl
Processing Batch 10518 (Rows 837070 - 837120)...
Finished batch 10518. Saved: pfdi_stats_batch_10518.pkl
Processing Batch 10519 (Rows 837120 - 837170)...
Finished batch 10519. Saved: pfdi_stats_batch_10519.pkl
Processing Batch 10520 (Rows 837170 - 837220)...
Finished batch 10520. Saved: pfdi_stats_batch_10520.pkl
Processing Batch 10521 (Rows 837220 - 837270)...
Finished batch 10521. Saved: pfdi_stats_batch_10521.pkl
Processing Batch 10522 (Rows 837270 - 837320)...
Finished batch 10522. Saved: pfdi_stats_batch_10522.pkl

Finished batch 10592. Saved: pfdi_stats_batch_10592.pkl
Processing Batch 10593 (Rows 840820 - 840870)...
Finished batch 10593. Saved: pfdi_stats_batch_10593.pkl
Processing Batch 10594 (Rows 840870 - 840920)...
Finished batch 10594. Saved: pfdi_stats_batch_10594.pkl
Processing Batch 10595 (Rows 840920 - 840970)...
Finished batch 10595. Saved: pfdi_stats_batch_10595.pkl
Processing Batch 10596 (Rows 840970 - 841020)...
Finished batch 10596. Saved: pfdi_stats_batch_10596.pkl
Processing Batch 10597 (Rows 841020 - 841070)...
Finished batch 10597. Saved: pfdi_stats_batch_10597.pkl
Processing Batch 10598 (Rows 841070 - 841120)...
Finished batch 10598. Saved: pfdi_stats_batch_10598.pkl
Processing Batch 10599 (Rows 841120 - 841170)...
Finished batch 10599. Saved: pfdi_stats_batch_10599.pkl
Processing Batch 10600 (Rows 841170 - 841220)...
Finished batch 10600. Saved: pfdi_stats_batch_10600.pkl
Processing Batch 10601 (Rows 841220 - 841270)...
Finished batch 10601. Saved: pfdi_stats_batch_10601.pkl

Finished batch 10671. Saved: pfdi_stats_batch_10671.pkl
Processing Batch 10672 (Rows 844770 - 844820)...
Finished batch 10672. Saved: pfdi_stats_batch_10672.pkl
Processing Batch 10673 (Rows 844820 - 844870)...
Finished batch 10673. Saved: pfdi_stats_batch_10673.pkl
Processing Batch 10674 (Rows 844870 - 844920)...
Finished batch 10674. Saved: pfdi_stats_batch_10674.pkl
Processing Batch 10675 (Rows 844920 - 844970)...
Finished batch 10675. Saved: pfdi_stats_batch_10675.pkl
Processing Batch 10676 (Rows 844970 - 845020)...
Finished batch 10676. Saved: pfdi_stats_batch_10676.pkl
Processing Batch 10677 (Rows 845020 - 845070)...
Finished batch 10677. Saved: pfdi_stats_batch_10677.pkl
Processing Batch 10678 (Rows 845070 - 845120)...
Finished batch 10678. Saved: pfdi_stats_batch_10678.pkl
Processing Batch 10679 (Rows 845120 - 845170)...
Finished batch 10679. Saved: pfdi_stats_batch_10679.pkl
Processing Batch 10680 (Rows 845170 - 845220)...
Finished batch 10680. Saved: pfdi_stats_batch_10680.pkl

Finished batch 10750. Saved: pfdi_stats_batch_10750.pkl
Processing Batch 10751 (Rows 848720 - 848770)...
Finished batch 10751. Saved: pfdi_stats_batch_10751.pkl
Processing Batch 10752 (Rows 848770 - 848820)...
Finished batch 10752. Saved: pfdi_stats_batch_10752.pkl
Processing Batch 10753 (Rows 848820 - 848870)...
Finished batch 10753. Saved: pfdi_stats_batch_10753.pkl
Processing Batch 10754 (Rows 848870 - 848920)...
Finished batch 10754. Saved: pfdi_stats_batch_10754.pkl
Processing Batch 10755 (Rows 848920 - 848970)...
Finished batch 10755. Saved: pfdi_stats_batch_10755.pkl
Processing Batch 10756 (Rows 848970 - 849020)...
Finished batch 10756. Saved: pfdi_stats_batch_10756.pkl
Processing Batch 10757 (Rows 849020 - 849070)...
Finished batch 10757. Saved: pfdi_stats_batch_10757.pkl
Processing Batch 10758 (Rows 849070 - 849120)...
Finished batch 10758. Saved: pfdi_stats_batch_10758.pkl
Processing Batch 10759 (Rows 849120 - 849170)...
Finished batch 10759. Saved: pfdi_stats_batch_10759.pkl

...

Finished batch 11066. Saved: pfdi_stats_batch_11066.pkl
Processing Batch 11067 (Rows 864520 - 864570)...
Finished batch 11067. Saved: pfdi_stats_batch_11067.pkl
Processing Batch 11068 (Rows 864570 - 864620)...
Finished batch 11068. Saved: pfdi_stats_batch_11068.pkl
Processing Batch 11069 (Rows 864620 - 864670)...
Finished batch 11069. Saved: pfdi_stats_batch_11069.pkl
Processing Batch 11070 (Rows 864670 - 864720)...
Finished batch 11070. Saved: pfdi_stats_batch_11070.pkl
Processing Batch 11071 (Rows 864720 - 864770)...
Finished batch 11071. Saved: pfdi_stats_batch_11071.pkl
Processing Batch 11072 (Rows 864770 - 864820)...
Finished batch 11072. Saved: pfdi_stats_batch_11072.pkl
Processing Batch 11073 (Rows 864820 - 864870)...
Finished batch 11073. Saved: pfdi_stats_batch_11073.pkl
Processing Batch 11074 (Rows 864870 - 864920)...
Finished batch 11074. Saved: pfdi_stats_batch_11074.pkl
Processing Batch 11075 (Rows 864920 - 864970)...
Finished batch 11075. Saved: pfdi_stats_batch_11075.pkl

Finished batch 11145. Saved: pfdi_stats_batch_11145.pkl
Processing Batch 11146 (Rows 868470 - 868520)...
Finished batch 11146. Saved: pfdi_stats_batch_11146.pkl
Processing Batch 11147 (Rows 868520 - 868570)...
Finished batch 11147. Saved: pfdi_stats_batch_11147.pkl
Processing Batch 11148 (Rows 868570 - 868620)...
Finished batch 11148. Saved: pfdi_stats_batch_11148.pkl
Processing Batch 11149 (Rows 868620 - 868670)...
Finished batch 11149. Saved: pfdi_stats_batch_11149.pkl
Processing Batch 11150 (Rows 868670 - 868720)...
Finished batch 11150. Saved: pfdi_stats_batch_11150.pkl
Processing Batch 11151 (Rows 868720 - 868770)...
Finished batch 11151. Saved: pfdi_stats_batch_11151.pkl
Processing Batch 11152 (Rows 868770 - 868820)...
Finished batch 11152. Saved: pfdi_stats_batch_11152.pkl
Processing Batch 11153 (Rows 868820 - 868870)...
Finished batch 11153. Saved: pfdi_stats_batch_11153.pkl
Processing Batch 11154 (Rows 868870 - 868920)...
Finished batch 11154. Saved: pfdi_stats_batch_11154.pkl

Finished batch 11224. Saved: pfdi_stats_batch_11224.pkl
Processing Batch 11225 (Rows 872420 - 872470)...
Finished batch 11225. Saved: pfdi_stats_batch_11225.pkl
Processing Batch 11226 (Rows 872470 - 872520)...
Finished batch 11226. Saved: pfdi_stats_batch_11226.pkl
Processing Batch 11227 (Rows 872520 - 872570)...
Finished batch 11227. Saved: pfdi_stats_batch_11227.pkl
Processing Batch 11228 (Rows 872570 - 872620)...
Finished batch 11228. Saved: pfdi_stats_batch_11228.pkl
Processing Batch 11229 (Rows 872620 - 872670)...
Finished batch 11229. Saved: pfdi_stats_batch_11229.pkl
Processing Batch 11230 (Rows 872670 - 872720)...
Finished batch 11230. Saved: pfdi_stats_batch_11230.pkl
Processing Batch 11231 (Rows 872720 - 872770)...
Finished batch 11231. Saved: pfdi_stats_batch_11231.pkl
Processing Batch 11232 (Rows 872770 - 872820)...
Finished batch 11232. Saved: pfdi_stats_batch_11232.pkl
Processing Batch 11233 (Rows 872820 - 872870)...
Finished batch 11233. Saved: pfdi_stats_batch_11233.pkl

Finished batch 11303. Saved: pfdi_stats_batch_11303.pkl
Processing Batch 11304 (Rows 876370 - 876420)...
Finished batch 11304. Saved: pfdi_stats_batch_11304.pkl
Processing Batch 11305 (Rows 876420 - 876470)...
Finished batch 11305. Saved: pfdi_stats_batch_11305.pkl
Processing Batch 11306 (Rows 876470 - 876520)...
Finished batch 11306. Saved: pfdi_stats_batch_11306.pkl
Processing Batch 11307 (Rows 876520 - 876570)...
Finished batch 11307. Saved: pfdi_stats_batch_11307.pkl
Processing Batch 11308 (Rows 876570 - 876620)...
Finished batch 11308. Saved: pfdi_stats_batch_11308.pkl
Processing Batch 11309 (Rows 876620 - 876670)...
Finished batch 11309. Saved: pfdi_stats_batch_11309.pkl
Processing Batch 11310 (Rows 876670 - 876720)...
Finished batch 11310. Saved: pfdi_stats_batch_11310.pkl
Processing Batch 11311 (Rows 876720 - 876770)...
Finished batch 11311. Saved: pfdi_stats_batch_11311.pkl
Processing Batch 11312 (Rows 876770 - 876820)...
Finished batch 11312. Saved: pfdi_stats_batch_11312.pkl

Finished batch 11382. Saved: pfdi_stats_batch_11382.pkl
Processing Batch 11383 (Rows 880320 - 880370)...
Finished batch 11383. Saved: pfdi_stats_batch_11383.pkl
Processing Batch 11384 (Rows 880370 - 880420)...
Finished batch 11384. Saved: pfdi_stats_batch_11384.pkl
Processing Batch 11385 (Rows 880420 - 880470)...
Finished batch 11385. Saved: pfdi_stats_batch_11385.pkl
Processing Batch 11386 (Rows 880470 - 880520)...
Finished batch 11386. Saved: pfdi_stats_batch_11386.pkl
Processing Batch 11387 (Rows 880520 - 880570)...
Finished batch 11387. Saved: pfdi_stats_batch_11387.pkl
Processing Batch 11388 (Rows 880570 - 880620)...
Finished batch 11388. Saved: pfdi_stats_batch_11388.pkl
Processing Batch 11389 (Rows 880620 - 880670)...
Finished batch 11389. Saved: pfdi_stats_batch_11389.pkl
Processing Batch 11390 (Rows 880670 - 880720)...
Finished batch 11390. Saved: pfdi_stats_batch_11390.pkl
Processing Batch 11391 (Rows 880720 - 880770)...
Finished batch 11391. Saved: pfdi_stats_batch_11391.pkl

In [5]:
df_enriched

,Unnamed: 0,uuid,area_km2,version,start_time,end_time,duration_days,geometry_wkt,urban_built_up_area_m2,polygon_total_area_m2,urban_percentage,event_id,poly_area_km2,upa_max,upa_p95,upa_p99,PFDI_p95,idx_p95
0,0,93e48341dbaa4be29a38b856d94206c5,191.436317,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-7.6844751 33.499789, -7.5516781 33....",4608358.0,1.531473e+08,3.009102,0,152.737360,2144.798828,0.231797,506.047577,0.001518,0.000010
1,1,939149831ea34c3e94712cc42c17c46a,14.208894,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-7.9863739 29.745153, -7.9965878 29....",751304.0,9.880934e+06,7.603573,1,9.846162,1930.907593,24.820149,1929.830884,2.520794,0.256018
2,2,931831f75479454cb3938811e64ce882,28.562733,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-98.97432499999999 19.337026, -98.93...",3757833.0,1.180029e+07,31.845253,2,11.730829,8171.314453,0.613581,0.613581,0.052305,0.004459
3,3,91bc68dde43646289c9e75b7d59e1711,108.562378,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((30.936502 -29.872152, 30.928104 -29....",26634641.0,1.273836e+08,20.908999,3,126.937144,4441.062012,0.377705,243.834593,0.002976,0.000023
4,4,8b849511de80423f98918f55af323e7e,0.104375,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((16.547385 38.687964, 16.549982 38.68...",49138.0,1.051849e+05,46.715826,4,0.105030,0.107175,0.073682,0.073682,0.701532,6.679364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882967,882967,7dcf66e5f2cb41aeabbca2eceb2a8097,170.996837,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((105.7873 21.524524, 105.70281 21.576...",13188502.0,1.484232e+08,8.885740,882967,147.693325,3142.486816,0.228462,2677.571888,0.001547,0.000010
882968,882968,315638fd86a7475faa9080ad49611dcf,47.735869,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((108.19657 15.435928, 108.21532 15.49...",104740.0,5.231977e+07,0.200192,882968,52.021208,49.398094,1.377442,18.209266,0.026478,0.000509
882969,882969,2a7732d881444a50b302459a74147b78,2.111778,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((110.4417 -6.949991, 110.44865 -6.938...",190181.0,2.413080e+06,7.881256,882969,2.397496,69.979530,57.835690,58.735450,24.123371,10.061902
882970,882970,728e991979374bcf8be0ceec27f83474,69.695840,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((107.57804 16.494362, 107.62911 16.46...",10160252.0,6.160880e+07,16.491560,882970,61.264040,1600.166748,0.151170,1586.783159,0.002468,0.000040


In [5]:

# ============================================
# 8) CONCAT ALL BATCH RESULTS (OLD AND NEW)
# ============================================
print("\n--- FINALIZING ---")
print("Merging all PKL files found in output directory (previous + current)...")

pkl_pattern = os.path.join(OUT_DIR, "pfdi_stats_batch_*.pkl")
all_pkl_files = glob.glob(pkl_pattern)

if not all_pkl_files:
    print("No PKL files found. Nothing to merge.")
else:
    print(f"Found {len(all_pkl_files)} batch files to merge.")

    # Read and concatenate
    df_pfdi_stats = pd.concat((pd.read_pickle(f) for f in all_pkl_files), ignore_index=True)

    # Remove duplicates (safety check)
    # Keep last in case newer runs overwrote/updated an event
    df_pfdi_stats = df_pfdi_stats.drop_duplicates(subset=["event_id"], keep="last")

    # Save combined stats PKL
    df_pfdi_stats.to_pickle(OUT_STATS_PKL_ALL)
    print("Saved combined stats table:", OUT_STATS_PKL_ALL)
    print(f"Total unique events processed so far: {len(df_pfdi_stats)}")

    # ============================================
    # 9) MERGE PFDI BACK INTO THE ORIGINAL PKL
    # ============================================
    print("Merging with original DataFrame...")

    df_orig = pd.read_pickle(PKL_PATH)
    df_orig = add_event_id(df_orig)

    df_enriched = df_orig.merge(df_pfdi_stats, on="event_id", how="left")

    # ============================================
    # 10) QC + CALCULATE PFDI INDICES (SAVE THIS VERSION)
    # ============================================
    print("Applying QC filters and computing PFDI indices...")

    # Replace inf with NaN
    df_enriched = df_enriched.replace([np.inf, -np.inf], np.nan)

    cols_to_check = ["upa_p95", "upa_p99", "upa_max", "poly_area_km2"]

    # Drop missing key columns, then filter for positive values
    df_enriched_clean = df_enriched.dropna(subset=cols_to_check).copy()

    df_enriched_clean = df_enriched_clean[
        (df_enriched_clean["upa_p95"] > 0) &
        (df_enriched_clean["upa_p99"] > 0) &
        (df_enriched_clean["upa_max"] > 0) &
        (df_enriched_clean["poly_area_km2"] > 0)
    ].copy()

    # Compute PFDI indices (define them explicitly from UPA / area)
    df_enriched_clean["PFDI_p95"] = df_enriched_clean["upa_p95"] / df_enriched_clean["poly_area_km2"]
    df_enriched_clean["PFDI_p99"] = df_enriched_clean["upa_p99"] / df_enriched_clean["poly_area_km2"]
    df_enriched_clean["PFDI_max"] = df_enriched_clean["upa_max"] / df_enriched_clean["poly_area_km2"]

    print(f"Rows before QC: {len(df_enriched)}")
    print(f"Rows after QC (positive values & no NaNs): {len(df_enriched_clean)}")

    # Save the final PKL (clean + computed)
    df_enriched_clean.to_pickle(OUT_ENRICHED_PKL)
    print("Saved FINAL enriched+clean PKL:", OUT_ENRICHED_PKL)

    # Validation print
    if "PFDI_p95" in df_enriched_clean.columns:
        count_valid = df_enriched_clean["PFDI_p95"].count()
        print(f"Enriched+clean PKL contains {count_valid} events with valid PFDI_p95 data.")
    else:
        print("Warning: PFDI_p95 column not found in final DataFrame.")


--- FINALIZING ---
Merging all PKL files found in output directory (previous + current)...
Found 11437 batch files to merge.
Saved combined stats table: D:\Development\RESEARCH\urban_flood_database\chronicle\hydromerit_pluvial_outputs\pfdi_stats_ALL_batches.pkl
Total unique events processed so far: 882972
Merging with original DataFrame...
Applying QC filters and computing PFDI indices...
Rows before QC: 882972
Rows after QC (positive values & no NaNs): 882957
Saved FINAL enriched+clean PKL: D:\Development\RESEARCH\urban_flood_database\chronicle\hydromerit_pluvial_outputs\chronicle_df_with_pfdi_FULL.pkl
Enriched+clean PKL contains 882957 events with valid PFDI_p95 data.


In [7]:
df_enriched_clean

,Unnamed: 0,uuid,area_km2,version,start_time,end_time,duration_days,geometry_wkt,urban_built_up_area_m2,polygon_total_area_m2,urban_percentage,event_id,poly_area_km2,upa_max,upa_p95,upa_p99,PFDI_p95,PFDI_p99,PFDI_max
0,0,93e48341dbaa4be29a38b856d94206c5,191.436317,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-7.6844751 33.499789, -7.5516781 33....",4608358.0,1.531473e+08,3.009102,0,152.737360,2144.798828,0.231797,506.047577,0.001518,3.313188,14.042398
1,1,939149831ea34c3e94712cc42c17c46a,14.208894,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-7.9863739 29.745153, -7.9965878 29....",751304.0,9.880934e+06,7.603573,1,9.846162,1930.907593,24.820149,1929.830884,2.520794,195.998295,196.107648
2,2,931831f75479454cb3938811e64ce882,28.562733,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((-98.97432499999999 19.337026, -98.93...",3757833.0,1.180029e+07,31.845253,2,11.730829,8171.314453,0.613581,0.613581,0.052305,0.052305,696.567534
3,3,91bc68dde43646289c9e75b7d59e1711,108.562378,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((30.936502 -29.872152, 30.928104 -29....",26634641.0,1.273836e+08,20.908999,3,126.937144,4441.062012,0.377705,243.834593,0.002976,1.920908,34.986308
4,4,8b849511de80423f98918f55af323e7e,0.104375,v3.1,9.466848e+08,9.466848e+08,1,"POLYGON ((16.547385 38.687964, 16.549982 38.68...",49138.0,1.051849e+05,46.715826,4,0.105030,0.107175,0.073682,0.073682,0.701532,0.701532,1.020426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882967,882967,7dcf66e5f2cb41aeabbca2eceb2a8097,170.996837,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((105.7873 21.524524, 105.70281 21.576...",13188502.0,1.484232e+08,8.885740,882967,147.693325,3142.486816,0.228462,2677.571888,0.001547,18.129268,21.277108
882968,882968,315638fd86a7475faa9080ad49611dcf,47.735869,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((108.19657 15.435928, 108.21532 15.49...",104740.0,5.231977e+07,0.200192,882968,52.021208,49.398094,1.377442,18.209266,0.026478,0.350035,0.949576
882969,882969,2a7732d881444a50b302459a74147b78,2.111778,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((110.4417 -6.949991, 110.44865 -6.938...",190181.0,2.413080e+06,7.881256,882969,2.397496,69.979530,57.835690,58.735450,24.123371,24.498663,29.188589
882970,882970,728e991979374bcf8be0ceec27f83474,69.695840,v3.1,1.761437e+09,1.761437e+09,1,"POLYGON ((107.57804 16.494362, 107.62911 16.46...",10160252.0,6.160880e+07,16.491560,882970,61.264040,1600.166748,0.151170,1586.783159,0.002468,25.900727,26.119184
